## Run Length Encoding Image

Task: https://www.codewars.com/kata/58bfa40c43fadb4edb0000b5

IONU Satellite Imaging, Inc. records and stores very large images using run length encoding. You are to write a program that reads a compressed image, finds the edges in the image, as described below, and outputs another compressed image of the detected edges.

A simple edge detection algorithm sets an output pixel's value to be the maximum absolute value of the differences between it and all its surrounding pixels in the input image. Consider the input image below:

The upper left pixel in the output image is the maximum of the values |15-15|,|15-100|, and |15-100|, which is 85. The pixel in the 4th row, 2nd column is computed as the maximum of |175-100|, |175-100|, |175-100|, |175-175|, |175-25|, |175-175|,|175-175|, and |175-25|, which is 150.

Images contain 2 to 1,000,000,000 (10^9) pixels. All images are encoded using run length encoding (RLE). This is a sequence of pairs, containing pixel value (0-255) and run length (1-10^9). Input images have at most 1,000 of these pairs. Successive pairs have different pixel values. All lines in an image contain the same number of pixels.

For the image as the example above, the RLE encoding string is "7 15 4 100 15 25 2 175 2 25 5 175 2 25 5"

 Each image starts with the width, in pixels(means the first number 7)
 This is followed by the RLE pairs(two number is a pair).
 
 
 7      ----> image width
 15 4   ----> a pair(color value + number of pixel)
 100 15  
 25 2         
 175 2        
 25 5         
 175 2        
 25 5         

### Solution:

In [1]:
import numpy as np

image = '7 15 4 100 15 25 2 175 2 25 5 175 2 25 5'

def reshape_image(image, r, c):
    # Adds duplicates of the edge rows so that the mesh grid can eliminate the values
    image = np.array([np.append(np.append([x[0]]*c[0], x), [x[-1]]*c[1]) for x in image])
    image_r, image_c = image.shape
    if r[0] == 0:
        return np.array(np.append(image, [image[-1]]*2)).reshape(image_r+2, image_c)
    elif r[0] == 2:
        return np.array(np.append([image[0]]*2, image)).reshape(image_r+2, image_c)
    else:
        return np.array(np.append(np.append(np.zeros(image_c), image), \
                                  np.zeros(image_c))).reshape(image_r+2, image_c)

def edge_detection(image):
    
    image_list = image.split(" ")
    width = int(image_list.pop(0))
    val = [int(i) for i in image_list[::2]]
    qt = [int(i) for i in image_list[1::2]]
    height = int(sum(qt)/width)
    
    # Redefine the matrix with 0's on the borders so that we can offset its interior
    # and perform subsequent matrix operations
    image_matrix_circumscribed = np.zeros((height+2, width+2))

    x, y = 0, 0
    # Decoder
    for num, times in zip(val, qt):
        for _ in range(times):
            image_matrix_circumscribed[y+1,x+1] = num
            x += 1
            if x == width:
                x = 0
                y += 1
    
    # The inners
    image_matrix = image_matrix_circumscribed[1:height+1,1:width+1].copy()
    
    mesh_grids = []
    for rofset in range(-1,2):
        add_above = np.abs(1+rofset)
        row_add = [add_above, 2-add_above]
        for cofset in range(-1,2):
            if rofset == 0 and cofset == 0: continue
            else:
                add_left = np.abs(1+cofset)
                col_add = [add_left, 2-add_left]
                # Reshape inners to fit shape image_matrix_circumscribed
                reshaped_image = reshape_image(image_matrix, row_add, col_add)
                mesh_grids.append((image_matrix_circumscribed-reshaped_image)[1:height+1,1:width+1])
    
    edge_matrix = np.zeros(image_matrix.shape)
    mesh_grids = np.array([np.abs(row) for row in mesh_grids])
    for row in range(height):
        for col in range(width):
            maximum = 0
            for value in mesh_grids:
                if value[row,col] > maximum: maximum = value[row,col]
            edge_matrix[row,col] = maximum
    
    # Encode
    string = str(width)
    last_val = edge_matrix[0][0]
    count = 0
    for val in edge_matrix.reshape(1,-1)[0]:
        if val != last_val:
            string += " " + str(int(last_val)) + " " + str(count)
            count = 1
        else:
            count += 1
        last_val = val
        
    string += " " + str(int(last_val)) + " " + str(count)
    print("Initial string: ", image)
    print("Initial image: \n", image_matrix, end = "\n\n")
    print("Final image: \n", edge_matrix)
    return string

edge_detection(image)

Initial string:  7 15 4 100 15 25 2 175 2 25 5 175 2 25 5
Initial image: 
 [[ 15.  15.  15.  15. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100.  25.  25.]
 [175. 175.  25.  25.  25.  25.  25.]
 [175. 175.  25.  25.  25.  25.  25.]]

Final image: 
 [[ 85.  85.  85.  85.  85.   0.   0.]
 [ 85.  85.  85.  85.  85.  75.  75.]
 [ 75.  75.  75.  75.  75.  75.  75.]
 [ 75. 150. 150.  75.  75.  75.   0.]
 [  0. 150. 150.   0.   0.   0.   0.]]


'7 85 5 0 2 85 5 75 10 150 2 75 3 0 2 150 2 0 4'